In [25]:
import os
import tensorflow as tf
import numpy as np
import keras
from tensorflow.keras.utils import to_categorical
from tensorflow.keras import models , layers
import cv2
import matplotlib.pyplot as plt
from tensorflow.keras.preprocessing.image import ImageDataGenerator, array_to_img, img_to_array, load_img
from tensorflow.keras.regularizers import l2
from tensorflow.keras.callbacks import ModelCheckpoint
from sklearn.metrics import classification_report, confusion_matrix

In [2]:
train_dir = "../face_mask_detection/DATADIR/Train"
valid_dir = "../face_mask_detection/DATADIR/Validation"
test_dir = "../face_mask_detection/DATADIR/Test"

In [3]:
tf.config.experimental.list_physical_devices()

[PhysicalDevice(name='/physical_device:CPU:0', device_type='CPU')]

In [4]:
train_datagen = ImageDataGenerator( rescale =1./255, zoom_range = 0.2, horizontal_flip = True)
training_set = train_datagen.flow_from_directory(train_dir,target_size = (64, 64),
batch_size = 32,class_mode = 'binary')


Found 10000 images belonging to 2 classes.


In [5]:
test_datagen = ImageDataGenerator(rescale = 1./255)
test_set = test_datagen.flow_from_directory(test_dir,
target_size = (64, 64),batch_size = 32, class_mode = 'binary')

Found 992 images belonging to 2 classes.


In [6]:
cnn = tf.keras.models.Sequential()

In [7]:
cnn.add(tf.keras.layers.Conv2D(filters=5, kernel_size=3, activation='relu', input_shape=[64,64,3]))

In [8]:
cnn.add(tf.keras.layers.MaxPooling2D(pool_size=2 , strides=2))

In [9]:
cnn.add(tf.keras.layers.Conv2D(filters=32, kernel_size=3, activation='relu'))
cnn.add(tf.keras.layers.MaxPool2D(pool_size=2, strides=2))

In [10]:
cnn.add(tf.keras.layers.Flatten())

In [11]:
cnn.add(tf.keras.layers.Dense(units=128 , activation='relu'))

In [12]:
cnn.compile(
    optimizer= 'adam' ,
    loss= 'binary_crossentropy',
    metrics= ['accuracy']
)

In [13]:
cnn.fit(x = training_set , validation_data= test_set , epochs=5)

Epoch 1/5
313/313 [==============================] - 838s 3s/step - loss: 2.9859 - accuracy: 0.0315 - val_loss: 2.8868 - val_accuracy: 0.0575

In [30]:
pred = cnn.predict(test_set)
pred

array([[0.       , 0.       , 0.       , ..., 0.       , 0.       ,
        0.       ],
       [0.       , 0.       , 0.       , ..., 0.       , 0.       ,
        0.       ],
       [0.       , 0.       , 0.       , ..., 0.       , 0.       ,
        0.       ],
       ...,
       [0.       , 1.3501966, 1.6614214, ..., 1.5913222, 1.52957  ,
        1.6483212],
       [0.       , 1.3576822, 1.6754427, ..., 1.459656 , 1.6757814,
        1.7169755],
       [0.       , 2.071549 , 2.3600876, ..., 2.097242 , 2.4198449,
        2.4181283]], dtype=float32)

In [31]:
cnn.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 62, 62, 5)         140       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 31, 31, 5)         0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 29, 29, 32)        1472      
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 14, 14, 32)        0         
_________________________________________________________________
flatten (Flatten)            (None, 6272)              0         
_________________________________________________________________
dense (Dense)                (None, 128)               802944    
Total params: 804,556
Trainable params: 804,556
Non-trainable params: 0
__________________________________________________

In [35]:
training_set.shuffle

True

In [36]:
test_set.shuffle


True

In [37]:
valid_datagen = ImageDataGenerator(rescale=1./255 ,zoom_range=0.2 , horizontal_flip=True )
valid_set = valid_datagen.flow_from_directory(valid_dir, target_size=(64,64) , batch_size=32 , class_mode= 'binary')

Found 800 images belonging to 2 classes.


In [38]:
import joblib #for save our model
import pickle

In [40]:
valid_pred = cnn.predict(valid_set)
valid_pred


array([[0.       , 1.2699113, 1.4740957, ..., 1.3881679, 1.7074715,
        1.5810815],
       [0.       , 1.1220461, 1.2486743, ..., 1.111841 , 1.3997768,
        1.4451523],
       [0.       , 0.       , 0.       , ..., 0.       , 0.       ,
        0.       ],
       ...,
       [0.       , 1.7853651, 2.0030913, ..., 1.7915268, 1.9105576,
        1.9236623],
       [0.       , 0.900418 , 1.1600652, ..., 1.0570326, 1.1012679,
        1.0698848],
       [0.       , 1.3233333, 1.3558507, ..., 1.3745004, 1.3237988,
        1.3000697]], dtype=float32)